In [1]:
import pandas as pd

In [2]:
# change settings to account for massive possible number if image_ids per play
pd.set_option('max_colwidth',1000)
pd.set_option('max_seq_items','none')

In [3]:
# read csvs as data frames
# cast as string to avoid float/NaN errors

plays = pd.read_csv('in/plays.csv').astype(str)
play_auth = pd.read_csv('in/authors_plays.csv').astype(str) 
play_char = pd.read_csv('in/characters_plays.csv').astype(str) 
play_img = pd.read_csv('in/plays_images.csv').astype(str) 
play_perf = pd.read_csv('in/performances.csv').astype(str)

In [4]:
# mild clean up
play_perf = play_perf.drop('data_id', 1).drop('code', 1).drop('production_id',1)

In [5]:
plays.head()

,play_id,label_ja,label_ja_sort,label_ka,sort_ja,label_eng,label_eng_sort,first_staged,reference
0,1,Onnakoroshi abura no jigoku,Onnakoroshi abura no jigoku,女殺油地獄,おんなころしあぶらのじごく,The Woman-Killer and the Hell of Oil,"Woman-Killer and the Hell of Oil, The",1721,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku February 1982 Performance Program"
1,2,Shinjū ten no Amijima,Shinju ten no Amijima,心中天網島,しんじゅうてんのあみじま,The Love Suicides at Amijima,"Love Suicides at Amijima, The",1720,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku October 1986 Performance Program"
2,3,Sonezaki shinjū,Sonezaki shinju,曽根崎心中,そねざきしんじゅう,The Love Suicides at Sonezaki,"Love Suicides at Sonezaki, The",1703,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Keene, Donald, Bunraku: The Art of the Japanese Puppet Theatre (Tokyo: Kodansha International, 1973); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku February 1984, February 1981 Performance Programs"
3,4,Shinjū Yoigōshin,Shinju Yoigoshin,心中宵庚申,しんじゅうよいごうしん,The Love Suicide of Hambei and Ochiyo,"Love Suicide of Hambei and Ochiyo, The",1722,"Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku May 1975, February 1982 Performance Program; Chikamatsu: 5 Late Plays, translated and annotated by C. Andrew Gerstle (New York: Columbia University Press, c2001)"
4,5,Somemoyō imose no kadomatsu,Somemoyo imose no kadomatsu,染模様妹背門松,そめもよういもせのかげまつ,The Love of Osome and Hisamatsu,"Love of Osome and Hisamatsu, The",1767,"Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku May 1976, November 1984 Performance Programs"


In [6]:
# multiValDataFrame takes a dataframe df, column name df_index and column name multival_key
# it merges records with the same df_index and updates multival_index to an array of all the multival_index values.
# E.g for a data frame plays with multiple character_id values per play_id, returns a data frame with one record 
# per play_id, and one array of character_ids per play.

def multiValDataFrame(df, df_index, multival_key):
    new_df = pd.DataFrame({})
    cols = df.columns
    id_list = df[df_index].unique()
    for i in id_list:
        temp_row = df.loc[df[df_index] == i ]
        vals = []
        for col in cols:
            vals.append(temp_row[:1][col].values[0])

        temp_dict = dict(zip(cols, vals))
        multi_id_list = list(temp_row[multival_key])
    
        temp_dict[multival_key] = multi_id_list
    
        new_df = new_df.append(temp_dict, ignore_index=True)
    return new_df

In [7]:
# merge and multival authors
plays = pd.merge(plays, play_auth, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'author_id')

In [8]:
# merge and multival characters
plays = pd.merge(plays, play_char, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'character_id')

In [9]:
# merge and multival images
plays = pd.merge(plays, play_img, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'image_id')

In [10]:
# merge and multival performances
plays = pd.merge(plays, play_perf, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'performance_id')

In [11]:
plays = plays.rename(columns = {'play_id':'id'})
plays = plays[['id','author_id','character_id','first_staged','image_id','label_eng','label_eng_sort','label_ja','label_ja_sort','label_ka','performance_id','reference','sort_ja']]

In [12]:
# export to csv
plays.to_csv('plays_exp.csv', encoding='utf8', index=False)

In [13]:
# export to json
plays.to_json('plays_exp.json', orient="records", force_ascii=False)

In [14]:
plays.head()

,id,author_id,character_id,first_staged,image_id,label_eng,label_eng_sort,label_ja,label_ja_sort,label_ka,performance_id,reference,sort_ja
0,1,[6],"[1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 2298, 2307, 2317, 2319]",1721,"[10777, 10778, 10779, 10780, 10781, 10782, 10783, 10784, 10785, 10786, 10787, 10788, 10789, 10892, 10893, 10894, 10895, 10896, 10897, 10898, 10899, 10900, 10901, 10902, 10903, 10904, 10905, 10906, 10907, 10908, 10909, 10910, 10911, 10912, 10915, 10916, 10917, 10918, 10919, 10920, 10921, 10922, 10923, 10924, 10925, 10926, 10927, 10928, 10929, 10930, 10931, 10932, 10933, 10934, 10935, 10936, 10937, 10938, 10939, 10940, 10941, 10942, 10943, 10944, 10945, 10946, 10947, 10948, 10949, 10950, 10951, 10952, 10953, 10954, 10955, 10956, 10957, 10958, 10959, 10960, 10961, 10962, 10963, 10964, 10965, 10966, 10967, 10968, 10980, 11356, 11357, 11358, 11359, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11376, 11377, 11378, 11379, 11380, 11381, 11382, 11383, 11384, 11385, 11386, 11387, 45061, 45089, 45095, 45100, 45101, 46082, 46084, 46086, 46092, 46250, 46251, 46252, 46253, 46254, 46255, 46256, 46282, 46283, 46284, 46285, 46286, 4...",The Woman-Killer and the Hell of Oil,"Woman-Killer and the Hell of Oil, The",Onnakoroshi abura no jigoku,Onnakoroshi abura no jigoku,女殺油地獄,"[99, 194, 287, 414, 556, 576, 651, 778, 881, 976]","Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku February 1982 Performance Program",おんなころしあぶらのじごく
1,2,[6],"[1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1701, 1878, 1879, 1880, 2146, 2147]",1720,"[10649, 10650, 10651, 10652, 10653, 10654, 10655, 10656, 10657, 10658, 10659, 10660, 10661, 10662, 10663, 10664, 10665, 10666, 10667, 10668, 10669, 10670, 10671, 10672, 10673, 10674, 10675, 10676, 10677, 10678, 10679, 10680, 10681, 10682, 10683, 10684, 10685, 10686, 10687, 10688, 10689, 10690, 10691, 10692, 10693, 10694, 10695, 10696, 10697, 10698, 10699, 10700, 10701, 10702, 10703, 10704, 10705, 10706, 10707, 10708, 10709, 10710, 10711, 10712, 10713, 10714, 10715, 10716, 10717, 10718, 10719, 10720, 10721, 10722, 10723, 10724, 10725, 10726, 10727, 10728, 10729, 10730, 10731, 10732, 10733, 10734, 10735, 10736, 10737, 10738, 10739, 10740, 10741, 10742, 10743, 10744, 10745, 10746, 10747, 10748, 10749, 10750, 10751, 10752, 10753, 10754, 10755, 10756, 10757, 10758, 10759, 10760, 10761, 10762, 10763, 10764, 10765, 10766, 10767, 10768, 10769, 10770, 10771, 10772, 10773, 10774, 10775, 10776, 45041, 45043, 45046, 45047, 45087, 45090, 45091, 45097, 45098, 45461, 45463, 45814, 45816, 45817, 4...",The Love Suicides at Amijima,"Love Suicides at Amijima, The",Shinjū ten no Amijima,Shinju ten no Amijima,心中天網島,"[110, 262, 315, 383, 457, 554, 558, 573, 780, 791, 913, 951, 954]","Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku October 1986 Performance Program",しんじゅうてんのあみじま
2,3,[6],"[1344, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 2127]",1703,"[10557, 10558, 10559, 10560, 10561, 10562, 10563, 10564, 10565, 10566, 10567, 10568, 10569, 10570, 10571, 10572, 10573, 10574, 10575, 10576, 10577, 10578, 10579, 10580, 10581, 10582, 10583, 10584, 10585, 10586, 10587, 10588, 10589, 10590, 10591, 10592, 10593, 10594, 10595, 10596, 10597, 10598, 10599, 10600, 10601, 10602, 10603, 10604, 10605, 10606, 10607, 10608, 10609, 10610, 10611, 10612, 10613, 10614, 10615, 10616, 10617, 106